In [0]:
%python
from pyspark.sql import SparkSession

# Create a Spark session with the desired configurations
#spark = SparkSession.builder \
#    .appName("MyApp") \
#   .config("spark.executor.memory", "8g") \
#   .config("spark.executor.memoryOverhead", "4g") \
#   .getOrCreate()

# Define the path to the Parquet file in the volume
parquet_file_path = "/Volumes/dbrgstgnc1/testdb/bronze/yellow_tripdata_2025-01.parquet"

# Read the Parquet file into a DataFrame
df = spark.read.parquet(parquet_file_path)

# Define the target Delta table location or name
delta_table_name = "dbrgstgnc1.testdb.bronze_nyctaxi"

# Write the DataFrame to a Delta table
df.write.format("delta").mode("overwrite").saveAsTable(delta_table_name)

# Optional: Verify the table was created
#display(spark.sql(f"SELECT * FROM {delta_table_name} LIMIT 10"))

In [0]:
%python
import dlt

# Retrieve the bronze table's storage path
bronze_location = spark.sql("DESCRIBE TABLE EXTENDED dbrgstgnc1.testdb.bronze_nyctaxi").filter("col_name = 'Location'").select("data_type").first()[0]

# Define the silver table as a DLT streaming table
@dlt.table(name="silver_nyctaxi")
def silver_nyctaxi():
    return spark.readStream.format("delta").load(bronze_location)